# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2c64f8d5e0>
├── 'a' --> tensor([[ 1.4000,  0.1934, -1.1863],
│                   [-1.5245, -0.9823,  1.6038]])
└── 'x' --> <FastTreeValue 0x7f2bcffda340>
    └── 'c' --> tensor([[ 1.0378,  0.5662, -0.1120,  1.6279],
                        [-0.9283,  0.3824, -0.8181, -0.8091],
                        [-1.9933, -0.2542, -0.4623, -0.2984]])

In [4]:
t.a

tensor([[ 1.4000,  0.1934, -1.1863],
        [-1.5245, -0.9823,  1.6038]])

In [5]:
%timeit t.a

58.3 ns ± 0.0534 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2c64f8d5e0>
├── 'a' --> tensor([[ 1.4342,  1.4370,  1.9175],
│                   [-0.4025, -1.2369, -1.4924]])
└── 'x' --> <FastTreeValue 0x7f2bcffda340>
    └── 'c' --> tensor([[ 1.0378,  0.5662, -0.1120,  1.6279],
                        [-0.9283,  0.3824, -0.8181, -0.8091],
                        [-1.9933, -0.2542, -0.4623, -0.2984]])

In [7]:
%timeit t.a = new_value

66.4 ns ± 0.0157 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.4000,  0.1934, -1.1863],
               [-1.5245, -0.9823,  1.6038]]),
    x: Batch(
           c: tensor([[ 1.0378,  0.5662, -0.1120,  1.6279],
                      [-0.9283,  0.3824, -0.8181, -0.8091],
                      [-1.9933, -0.2542, -0.4623, -0.2984]]),
       ),
)

In [10]:
b.a

tensor([[ 1.4000,  0.1934, -1.1863],
        [-1.5245, -0.9823,  1.6038]])

In [11]:
%timeit b.a

54.5 ns ± 0.0236 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3504, -0.4902, -0.3609],
               [-0.5849, -1.3304,  0.3532]]),
    x: Batch(
           c: tensor([[ 1.0378,  0.5662, -0.1120,  1.6279],
                      [-0.9283,  0.3824, -0.8181, -0.8091],
                      [-1.9933, -0.2542, -0.4623, -0.2984]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.0974 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

883 ns ± 4.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 55.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

140 µs ± 411 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2bc8c90820>
├── 'a' --> tensor([[[ 1.4000,  0.1934, -1.1863],
│                    [-1.5245, -0.9823,  1.6038]],
│           
│                   [[ 1.4000,  0.1934, -1.1863],
│                    [-1.5245, -0.9823,  1.6038]],
│           
│                   [[ 1.4000,  0.1934, -1.1863],
│                    [-1.5245, -0.9823,  1.6038]],
│           
│                   [[ 1.4000,  0.1934, -1.1863],
│                    [-1.5245, -0.9823,  1.6038]],
│           
│                   [[ 1.4000,  0.1934, -1.1863],
│                    [-1.5245, -0.9823,  1.6038]],
│           
│                   [[ 1.4000,  0.1934, -1.1863],
│                    [-1.5245, -0.9823,  1.6038]],
│           
│                   [[ 1.4000,  0.1934, -1.1863],
│                    [-1.5245, -0.9823,  1.6038]],
│           
│                   [[ 1.4000,  0.1934, -1.1863],
│                    [-1.5245, -0.9823,  1.6038]]])
└── 'x' --> <FastTreeValue 0x7f2bc8c90490>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33 µs ± 40.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2c64fb80d0>
├── 'a' --> tensor([[ 1.4000,  0.1934, -1.1863],
│                   [-1.5245, -0.9823,  1.6038],
│                   [ 1.4000,  0.1934, -1.1863],
│                   [-1.5245, -0.9823,  1.6038],
│                   [ 1.4000,  0.1934, -1.1863],
│                   [-1.5245, -0.9823,  1.6038],
│                   [ 1.4000,  0.1934, -1.1863],
│                   [-1.5245, -0.9823,  1.6038],
│                   [ 1.4000,  0.1934, -1.1863],
│                   [-1.5245, -0.9823,  1.6038],
│                   [ 1.4000,  0.1934, -1.1863],
│                   [-1.5245, -0.9823,  1.6038],
│                   [ 1.4000,  0.1934, -1.1863],
│                   [-1.5245, -0.9823,  1.6038],
│                   [ 1.4000,  0.1934, -1.1863],
│                   [-1.5245, -0.9823,  1.6038]])
└── 'x' --> <FastTreeValue 0x7f2c64fb80a0>
    └── 'c' --> tensor([[ 1.0378,  0.5662, -0.1120,  1.6279],
                        [-0.9283,  0.3824, -0.8181, -0.8091],
                 

In [23]:
%timeit t_cat(trees)

31 µs ± 55.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.2 µs ± 53.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.4000,  0.1934, -1.1863],
                [-1.5245, -0.9823,  1.6038]],
       
               [[ 1.4000,  0.1934, -1.1863],
                [-1.5245, -0.9823,  1.6038]],
       
               [[ 1.4000,  0.1934, -1.1863],
                [-1.5245, -0.9823,  1.6038]],
       
               [[ 1.4000,  0.1934, -1.1863],
                [-1.5245, -0.9823,  1.6038]],
       
               [[ 1.4000,  0.1934, -1.1863],
                [-1.5245, -0.9823,  1.6038]],
       
               [[ 1.4000,  0.1934, -1.1863],
                [-1.5245, -0.9823,  1.6038]],
       
               [[ 1.4000,  0.1934, -1.1863],
                [-1.5245, -0.9823,  1.6038]],
       
               [[ 1.4000,  0.1934, -1.1863],
                [-1.5245, -0.9823,  1.6038]]]),
    x: Batch(
           c: tensor([[[ 1.0378,  0.5662, -0.1120,  1.6279],
                       [-0.9283,  0.3824, -0.8181, -0.8091],
                       [-1.9933, -0.2542, -0.4623, -0.2984]],
         

In [26]:
%timeit Batch.stack(batches)

78.6 µs ± 122 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.4000,  0.1934, -1.1863],
               [-1.5245, -0.9823,  1.6038],
               [ 1.4000,  0.1934, -1.1863],
               [-1.5245, -0.9823,  1.6038],
               [ 1.4000,  0.1934, -1.1863],
               [-1.5245, -0.9823,  1.6038],
               [ 1.4000,  0.1934, -1.1863],
               [-1.5245, -0.9823,  1.6038],
               [ 1.4000,  0.1934, -1.1863],
               [-1.5245, -0.9823,  1.6038],
               [ 1.4000,  0.1934, -1.1863],
               [-1.5245, -0.9823,  1.6038],
               [ 1.4000,  0.1934, -1.1863],
               [-1.5245, -0.9823,  1.6038],
               [ 1.4000,  0.1934, -1.1863],
               [-1.5245, -0.9823,  1.6038]]),
    x: Batch(
           c: tensor([[ 1.0378,  0.5662, -0.1120,  1.6279],
                      [-0.9283,  0.3824, -0.8181, -0.8091],
                      [-1.9933, -0.2542, -0.4623, -0.2984],
                      [ 1.0378,  0.5662, -0.1120,  1.6279],
                      [-0.9283,  

In [28]:
%timeit Batch.cat(batches)

145 µs ± 220 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 2.18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
